In [ ]:
import time

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np

import os, sys

import utils

from best_utils import *
from cross_validation import cross_validation
import image_preprocessing

from skimage import color

import keras

from skimage.color import rgb2hsv

In [ ]:
ROOT_DIR = 'training/'
IMAGE_DIR = ROOT_DIR + 'images/'

PATCH_SIZE = 8

N = 1 # Number of image to be used in training

In [ ]:
pad = lambda im: reflect_padding(im, 5)

In [ ]:
# Read images, change to hsv, saturate and stack into a tensor
files = os.listdir(IMAGE_DIR)
#imgs = np.stack([image_preprocessing.saturate_hsv_img(color.rgb2hsv(load_image(IMAGE_DIR + file))) for file in files]) # images (400, 400, 3)
imgs = np.stack([load_image(IMAGE_DIR + file) for file in files[:N]]) # images (400, 400, 3)


# Read groundtruts into a tensor and threshold their values
GT_DIR = ROOT_DIR + 'groundtruth/'
gt_imgs = np.stack([(load_image(GT_DIR + file)) for file in files[:N]]) # images (400, 400)

# Whole image feeding

In [ ]:
# Patch images and groundtruths
patched_imgs = np.stack([(patch_image(im, PATCH_SIZE)) for im in imgs])
patched_gts = np.stack([patch_groundtruth(gt, PATCH_SIZE) for gt in gt_imgs])

In [ ]:
X = np.stack(patched_imgs)
Y = np.expand_dims(np.stack(patched_gts), axis=-1)

In [ ]:
plt.imshow(X[0])

In [ ]:
plt.imshow(Y[0,:,:,0])

In [ ]:
X.shape, Y.shape

In [ ]:
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, LeakyReLU

In [ ]:
leakyness = 0.1

In [ ]:
deep_cnn = keras.models.Sequential([
    
    Conv2D(32, (5, 5), strides=(1, 1), input_shape=X.shape[1:]),
    LeakyReLU(leakyness),

    #keras.layers.Dropout(0.25),
    
    Conv2D(64, (3, 3), strides=(1, 1)),
    LeakyReLU(leakyness),

    #keras.layers.Dropout(0.25),
    
    Conv2D(128, (3, 3), strides=(1, 1)),
    LeakyReLU(leakyness),
    
    Conv2D(256, (3, 3), strides=(1, 1)),
    LeakyReLU(leakyness),

    #keras.layers.Dropout(0.25),
    
    #keras.layers.Dropout(0.25),

    Dense(256),
    LeakyReLU(leakyness),
    
    Dense(1),#, activation='softmax'),
    LeakyReLU(leakyness),
    
])

In [ ]:
deep_cnn.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(lr=0.00001), metrics=['accuracy'])

In [ ]:
deep_cnn.fit(X, Y, epochs=80)

In [ ]:
deep_cnn.summary()

In [ ]:
preds = deep_cnn.predict(X)

In [ ]:
preds.shape

In [ ]:
preds = np.squeeze(preds[0])

In [ ]:
preds.shape

In [ ]:
plt.plot(np.sort(preds[0]))

In [ ]:
np.min(preds[0]), np.max(preds[0])

In [ ]:
#plt.figure(figsize=(10,10))
plt.imshow(preds, cmap='Greys_r')

In [ ]:
plt.imshow(Y[0,:,:,0], cmap='Greys_r')

# Sliding window

In [ ]:
PATCH_SIZE = 10
WINDOW_SIZE = 55

In [ ]:
im, gt = patch_image(imgs[0], PATCH_SIZE), patch_groundtruth(gt_imgs[0], PATCH_SIZE)

In [ ]:
im.shape, gt.shape

obtain windows

In [ ]:
f = image_to_features(im, WINDOW_SIZE+10, True)

In [ ]:
f.shape

In [ ]:
windows = np.reshape(f, (f.shape[0], WINDOW_SIZE+10, WINDOW_SIZE+10, 3)) # 1 pic per pixel

In [ ]:
plt.imshow(reassemble(f, WINDOW_SIZE+10, 40, 40))

In [ ]:
#windows = np.stack([pad(w) for w in windows])

In [ ]:
windows.shape

In [ ]:
window_labels = np.ravel(gt)

In [ ]:
window_cnn = keras.models.Sequential([
    
    Conv2D(32, (5, 5), strides=(1, 1), input_shape=windows.shape[1:]),
    LeakyReLU(leakyness),
    
    MaxPooling2D(2),

    Conv2D(64, (3, 3), strides=(1, 1)),
    LeakyReLU(leakyness),
    
    MaxPooling2D(2),
    
    Conv2D(128, (3, 3), strides=(1, 1)),
    LeakyReLU(leakyness),
    
    MaxPooling2D(2),
    
    Conv2D(256, (3, 3), strides=(1, 1)),
    LeakyReLU(leakyness),

    MaxPooling2D(2),
    
    Dense(128),
    LeakyReLU(leakyness),
    
    Flatten(),
    Dense(1),#, activation='softmax'),
    LeakyReLU(leakyness),
])

In [ ]:
window_cnn.summary()

In [ ]:
windows.shape

In [ ]:
window_cnn.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(lr=0.0001), metrics=['accuracy'])

In [ ]:
nw = windows.shape[0]
def batch_generator():
    indices = np.random.choice(range(nw), 100, replace=False)
    for _ in range(16):
        yield windows[indices]

In [ ]:
#window_cnn.fit(windows, window_labels, epochs=20)
window_cnn.fit_generator(batch_generator(), )

In [ ]:
preds = window_cnn.predict(windows)

In [ ]:
windows.shape

In [ ]:
preds.shape

In [ ]:
f = np.ravel(preds)
f = np.reshape(f, (40, 40))

In [ ]:
plt.plot(np.sort(np.ravel(f)))

In [ ]:
f [f < 0.5] = 0
f [f >= 0.5] = 1

In [ ]:
f.shape, gt.shape

In [ ]:
plt.imshow(gt)

In [ ]:
plt.imshow(f)

In [ ]:
plt.imshow(np.hstack([gt, f]), cmap='Greys_r')